## Импорт библиотек и данных ##

In [1]:
import pandas as pd
import numpy as np
import os

import re

import torch
import faiss

from sentence_transformers import SentenceTransformer

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PATH_DATA = 'data'

TRANSFORMER = 'LaBSE-ru-turbo'
SENTENCE_MODEL = SentenceTransformer(f'sergeyzh/{TRANSFORMER}')

In [3]:
os.listdir(PATH_DATA)

['embeds_LaBSE-ru-turbo.csv', 'Примерное написание.csv', 'Школы.csv']

In [4]:
approx_df = pd.read_csv(f'{PATH_DATA}/Примерное написание.csv')
schools = pd.read_csv(f'{PATH_DATA}/Школы.csv')#, header=0)

In [6]:
display(approx_df.shape)
display(schools.shape)

(895, 2)

(306, 3)

Таблицы маленькие, поэтому можно безболезненно не ограничивать число выводимых строк началом и концом таблицы

In [5]:
approx_df

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""
...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор"
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,2,"ЯНАО, СШ ""Авангард"""


In [6]:
approx_df.sort_values('school_id')

,school_id,name
894,1,"Московская область, СШ ""Авангард"""
893,2,"ЯНАО, СШ ""Авангард"""
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
888,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор"""
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
...,...,...
4,609,"""СШ ""Гвоздика"""
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
2,610,"СШОР ""Надежда Губернии"
1,1836,"Москва, СК ""Триумф"""


In [7]:
# упорядочим значения для удобства
approx_df = approx_df.sort_values('school_id').reset_index(drop=True)

In [8]:
approx_df

,school_id,name
0,1,"Московская область, СШ ""Авангард"""
1,2,"ЯНАО, СШ ""Авангард"""
2,3,"СШОР ФСО Авиатор, Республика Татарстан"
3,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор"""
4,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
...,...,...
890,609,"""СШ ""Гвоздика"""
891,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
892,610,"СШОР ""Надежда Губернии"
893,1836,"Москва, СК ""Триумф"""


In [8]:
schools

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург
...,...,...,...
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край


In [10]:
schools['region'] = schools['region'].apply(lambda x: x.lower())

In [11]:
schools

,school_id,name,region
0,1,Авангард,московская область
1,2,Авангард,ямало-ненецкий ао
2,3,Авиатор,республика татарстан
3,4,Аврора,санкт-петербург
4,5,Ice Dream / Айс Дрим,санкт-петербург
...,...,...,...
301,305,Прогресс,алтайский край
302,609,"""СШ ""Гвоздика""",удмуртская республика
303,610,"СШОР ""Надежда Губернии",саратовская область
304,611,КФК «Айсберг»,пермский край


In [12]:
schools.query('region == "санкт-петербург"').tail(30)

,school_id,name,region
221,222,Темпо,санкт-петербург
222,223,ТИТУЛ,санкт-петербург
228,229,Фаворит,санкт-петербург
249,250,ЦОП по фигурному катанию на коньках,санкт-петербург
252,253,ЦФКСиЗ Красногвардейского района,санкт-петербург
253,254,ЦФКСиЗ Василеостровского района,санкт-петербург
254,256,ЦФКСиЗ Московского района,санкт-петербург
255,257,ЦФКСиЗ Фрунзенского района,санкт-петербург
257,259,Чемпион,санкт-петербург
267,270,ЦФКСиЗ Невского района,санкт-петербург


In [13]:
schools.groupby('region').agg('count').sort_values('name')

,school_id,name
region,,
алтайский край,1,1
липецкая область,1,1
набережные челны,1,1
новгородская область,1,1
омская область,1,1
...,...,...
республика татарстан,9,9
свердловская область,10,10
московская область,15,15


### Выводы по первичному анализу ###

1) Дубликатов не обнаружено: датасет подготовлен к построению модели;<br>
2) Больше всего школ в СПб, Москве и МО. Мы не можем проверить, все ли существующие присутствуют в нашей таблице, поэтому будем считать, что это так;<br>
3) Обучающий датасет - "Школы", таргет - school_id.</br>
4) Учитывая размеры таблиц, обучать с нуля не имеет никакого смысла, поэтому будет использован фреймворк SentenceTransformer. Модель LaBSE, так как он хорошо ищет пары.

## Подготовка данных ##

In [14]:
# фильтрация
def cleaning_text(description):
    text_lower = description.lower()
    clean = re.sub(r'([$: г ]\W)', ' ', text_lower)
    clean = re.sub(r'\W', ' ', clean)
    text_cleaned = clean.strip()
    return text_cleaned

In [15]:
schools['name'] = schools['name'].apply(cleaning_text)

In [16]:
approx_df['name'] = approx_df['name'].apply(cleaning_text)

In [17]:
approx_df

,school_id,name
0,1,московская область сш авангард
1,2,янао сш авангард
2,3,сшор фсо авиатор республика татарстан
3,3,республика татарстан мбу сшор фсо авиатор
4,3,республика татарстан мбу до сшор фсо авиатор
...,...,...
890,609,сш гвоздика
891,610,саратовская область гбусо сшор надежда губернии
892,610,сшор надежда губернии
893,1836,москва ск триумф


In [18]:
def create_or_load_text_embeddings(df, text_col_names, file_name=None, file_path=PATH_DATA, file_ext='csv'):

    file_name = f'{file_name}.{file_ext}'
    full_file_name = os.path.join(file_path, file_name)

    # если эмбединги извлечены и сохранены ранее
    if os.path.exists(full_file_name):
        if file_ext == 'csv':
            df_text_embeddings = pd.read_csv(full_file_name)
    
    else:
        df['concatenated_text'] = df.apply(lambda row: '. '.join(row.values.astype(str)), axis=1)
        text_transformer = SENTENCE_MODEL
        df_text_embeddings = text_transformer.encode(
                                                     df['concatenated_text'],
                                                     show_progress_bar=True,
                                                     device=DEVICE,
                                                     batch_size=32,
                                                    )

        df_text_embeddings = pd.DataFrame(df_text_embeddings)
        
        if file_ext == 'csv':
            df_text_embeddings.to_csv(full_file_name, index=False)
    
    return df_text_embeddings

In [19]:
%%time

embeds = create_or_load_text_embeddings(
                                        schools,
                                        text_col_names='name',
                                        file_name=f'embeds_{TRANSFORMER}',
                                        file_ext='csv'
                                       )

CPU times: total: 46.9 ms
Wall time: 70 ms


In [20]:
def create_text_embedding(text: str):

    text_transformer = SENTENCE_MODEL
    df_text_embeddings = text_transformer.encode(
                                                 text,
                                                 show_progress_bar=True,
                                                 device=DEVICE,
                                                 batch_size=32,
                                                )
    
    # df_text_embeddings = pd.DataFrame(df_text_embeddings)                                      # преобразование массива в датафрейм

    return df_text_embeddings

## Извлечение эмбеддингов и проверка работы ##

In [22]:
request_text = "гвзд"

request_embeds = create_text_embedding(request_text)       # извлечение эмбеддингов из текста запроса
request_embeds = np.atleast_2d(request_embeds)             # преобразование в двумерный массив

request_embeds.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)

In [23]:
index = faiss.IndexFlatL2(embeds.shape[1])
index.add(np.ascontiguousarray(embeds))

In [24]:
distances, indices = index.search(np.ascontiguousarray(request_embeds), 5)

In [27]:
df_1 = pd.DataFrame(distances).T.rename(columns={0: 'distance'})     # данные о расстояниях
df_2 = schools.loc[indices.flatten(), :].reset_index(drop=True)      # данные из исходной таблицы (можно добавить фильтрацию столбцов)

result_df = pd.concat([df_1, df_2], axis=1)
result_df

,distance,school_id,name,region
0,0.878496,609,сш гвоздика,удмуртская республика
1,0.915050,294,гбу мосспортобъект лц звезда,москва
2,0.916340,239,ффкк рк,республика коми
3,0.922975,37,гбоу до но сшор по лвс,нижегородская область
4,0.929736,200,сшор по фкк,республика мордовия


## Выводы по проекту ##

Мы получили рабочий метод по отбору наиболее релевантного названия для вводимого запроса, который при проверке на разных значениях выдаёт стабильные результаты.

Больше проектов можно найти на моём [Github](https://github.com/Vasart-ds). 
Связаться со мной и задать вопросы можно удобным способом:
* [Telegram](https://t.me/vasyukhin_art)
* [VK](https://vk.com/kvasart)
* [LinkedIn](https://www.linkedin.com/in/artem-vasyukhin-963126250/)